# import library

In [1]:
import torch
import torchvision

import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from model.edsr import *
import numpy as np

In [2]:
import easydict
args = easydict.EasyDict({
         "n_threads": 8,
 
        "n_GPUs": 1,
 
        "seed": 1,
 
        "scale": (2,),
    "n_colors": 3,
    "act": "Relu",
    
    "n_resblocks": 32,
    "n_feats": 256,
    "shift_mean": True,
    'res_scale':0.1,
    
        "rgb_range": 255
 })




In [3]:
model=EDSR(args)

In [4]:
m=model.head[0]

In [5]:
m.kernel_size

(3, 3)

In [6]:
m.__class__

torch.nn.modules.conv.Conv2d

In [7]:
def test():
    model = EDSR(args)
    print("\nmodel.head[0].weight.data\n",model.head[0].weight.data)
    print('\nmodel.head[0].__class__\n',model.head[0].__class__)
    print('\nmodel.head[0].kernel_size\n',model.head[0].kernel_size)
    input = torch.autograd.Variable(torch.randn(32, 3, 224, 224))
    print('\ninput.size()\n',input.size())
    print('\nnp.prod(input.size()): ',np.prod(input.size()))
test()


model.head[0].weight.data
 tensor([[[[-0.0797,  0.1522,  0.0095],
          [-0.1535, -0.0549,  0.0466],
          [ 0.0059,  0.0502,  0.0878]],

         [[-0.1682,  0.0583, -0.1736],
          [-0.0798, -0.1066, -0.0185],
          [ 0.1219, -0.0951, -0.0548]],

         [[-0.0825,  0.1706,  0.0973],
          [-0.1521, -0.0544, -0.0818],
          [ 0.0547,  0.1574, -0.1634]]],


        [[[-0.1446,  0.1351,  0.0315],
          [-0.1477,  0.0923, -0.1474],
          [ 0.0526, -0.0746,  0.0264]],

         [[ 0.0806,  0.0047,  0.0012],
          [ 0.1177,  0.0417,  0.1169],
          [ 0.0495,  0.0475,  0.0386]],

         [[-0.0075,  0.0476,  0.0706],
          [ 0.0819, -0.0520,  0.0678],
          [-0.0820,  0.0968, -0.1467]]],


        [[[-0.0119, -0.1075,  0.1160],
          [-0.0797,  0.0329, -0.0748],
          [ 0.1316,  0.1293,  0.0063]],

         [[-0.0815, -0.0003,  0.1885],
          [-0.1303, -0.1110,  0.0303],
          [-0.0145,  0.1523, -0.1371]],

         [[ 0.05

In [8]:
def print_model_parm_nums(my_models=models.resnet50()):
    model = my_models#EDSR(args)
    trainable = filter(lambda x: x.requires_grad, model.parameters())
    num_params = sum([np.prod(p.size()) for p in filter(lambda x: x.requires_grad, model.parameters())])
    print('num of parameters: %.2fM ' % (int(num_params) / 1e6) +'\n')
    print('estimated of size %.2fM ' % (int(num_params) / 1e6*4) +'\n')
print_model_parm_nums()

num of parameters: 25.56M 

estimated of size 102.23M 



In [9]:
#from model.edsr import *
from model.edsr_mobile import *
def print_model_parm_flops(my_models=models.resnet50()):
    multiply_adds = False
    list_conv=[]
    def conv_hook(self, input, output):
        Tt=lambda x:torch.tensor(x)
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
        kernel_ops = Tt(self.kernel_size[0]) * Tt(self.kernel_size[1]) * Tt(int(self.in_channels / self.groups))* Tt(2 if multiply_adds else 1)
        bias_ops = Tt(1 if self.bias is not None else 0)
        params = Tt(output_channels) * Tt(kernel_ops + bias_ops)
        flops = Tt(batch_size) * Tt(params) * Tt(output_height) * Tt(output_width)
        list_conv.append(flops)
    list_linear=[]
    def linear_hook(self, input, output):
        
        Tt=lambda x:torch.tensor(x)
        batch_size = Tt(input[0].size(0) if input[0].dim() == 2 else 1)

        weight_ops = Tt(self.weight.nelement()) * Tt(2 if multiply_adds else 1)
        bias_ops = Tt(self.bias.nelement())
        flops = Tt(batch_size) * Tt(weight_ops + bias_ops)
        list_linear.append(flops)
    list_bn=[]
    def bn_hook(self, input, output):
        list_bn.append(input[0].nelement())
        

    list_relu=[]
    def relu_hook(self, input, output):
        list_relu.append(input[0].nelement())

    list_pooling=[]
    def pooling_hook(self, input, output):
        Tt=lambda x:torch.tensor(x)
        batch_size, input_channels, input_height, input_width = input[0].size()
        output_channels, output_height, output_width = output[0].size()
        kernel_ops =Tt(self.kernel_size) * Tt(self.kernel_size)
        bias_ops = 0
        params = Tt(output_channels) * Tt(kernel_ops + bias_ops)
        flops = Tt(batch_size) * Tt(params) * Tt(output_height) * Tt(output_width)
        list_pooling.append(flops)



            
    def register_hook(net):
        childrens = list(net.children())
        if not childrens:
            if isinstance(net, torch.nn.Conv2d):
                net.register_forward_hook(conv_hook)
            if isinstance(net, torch.nn.Linear):
                net.register_forward_hook(linear_hook)
            if isinstance(net, torch.nn.BatchNorm2d):
                net.register_forward_hook(bn_hook)
            if isinstance(net, torch.nn.ReLU):
                net.register_forward_hook(relu_hook)
            if isinstance(net, torch.nn.MaxPool2d) or isinstance(net, torch.nn.AvgPool2d):
                net.register_forward_hook(pooling_hook)
            return
        for c in childrens:
                register_hook(c)

    resnet =my_models
     #EDSR(args)
    register_hook(resnet)
    with torch.no_grad():
        input = torch.rand(1,3,224,224)
        out = resnet(input)



    total_flops = (sum(list_conv) + sum(list_linear) + sum(list_bn) + sum(list_relu) + sum(list_pooling))
    
    print('Number of FLOPs: %.2fG' % (total_flops / 1e9)+'\n')
mo=EDSR(args)
print_model_parm_flops(mo)
print_model_parm_nums(mo)

Number of FLOPs: 356.00G

num of parameters: 6.82M 

estimated of size 27.27M 

